<a href="https://colab.research.google.com/github/linamarwabelkilani/modelisation-hydrologique/blob/main/netoyage_des_donn%C3%A9es_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing liibrarys 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from google.colab import files
from numpy import log as ln

In [4]:
#import dataset from github
data_url = "https://raw.githubusercontent.com/linamarwabelkilani/modelisation-hydrologique/main/Dataset.csv"
df = pd.read_csv(data_url, on_bad_lines='skip',sep=";")


In [6]:
df['Date'] = pd.to_datetime(df['Date'],dayfirst=True,errors='coerce')
df = df.dropna()
df.reset_index(drop=True, inplace=True)
df

,Date,Station,Precipitation
0,1951-06-01,AIN BEYA OUED RHEZAL,0.0
1,1951-06-02,AIN BEYA OUED RHEZAL,0.0
2,1951-06-03,AIN BEYA OUED RHEZAL,0.0
3,1951-06-04,AIN BEYA OUED RHEZAL,0.0
4,1951-06-05,AIN BEYA OUED RHEZAL,0.0
...,...,...,...
2103688,2015-08-27,ZOUARINE GARE,3.0
2103689,2015-08-28,ZOUARINE GARE,7.0
2103690,2015-08-29,ZOUARINE GARE,0.0
2103691,2015-08-30,ZOUARINE GARE,0.0


In [7]:
#supprission des précepitation avec valeur > 300 (3 ligne)
df = df.drop(df[df.Precipitation > 300].index)
df

,Date,Station,Precipitation
0,1951-06-01,AIN BEYA OUED RHEZAL,0.0
1,1951-06-02,AIN BEYA OUED RHEZAL,0.0
2,1951-06-03,AIN BEYA OUED RHEZAL,0.0
3,1951-06-04,AIN BEYA OUED RHEZAL,0.0
4,1951-06-05,AIN BEYA OUED RHEZAL,0.0
...,...,...,...
2103688,2015-08-27,ZOUARINE GARE,3.0
2103689,2015-08-28,ZOUARINE GARE,7.0
2103690,2015-08-29,ZOUARINE GARE,0.0
2103691,2015-08-30,ZOUARINE GARE,0.0


In [63]:
def percent(num_a, num_b):
  q=num_a / num_b
  return q * 100
count_zero=(df["Precipitation"] == 0).sum()
print("les nombres des zero : ",count_zero , " pourcontage : ",percent(count_zero,len(df.index)),"%")
# 82.9 % des valeur dans notre Dataset sont des zeros !! 

les nombres des zero :  1745539  pourcontage :  82.97510564769523 %


In [81]:
station=df.groupby(["Station"]).sum()
print("nombre des stations : ", len(station.index))
#groupby chaque station et count les ligne ou la valeur de Precipitation et = 0 
nbr_zero_par_station=df.groupby('Station')['Precipitation'].apply(lambda x: (x==0).sum()).reset_index(name='Precipitation_zero')
nbr_zero_par_station["Pourcentage par rapport au nbr zero total%"] = nbr_zero_par_station.apply(lambda x:percent(x["Precipitation_zero"],len(df.index)),axis=1)
#dataframe avec les nombres des zero par chaque station par ordre décroissante 
nbr_zero_par_station.sort_values(by=["Precipitation_zero"],ascending=False, inplace=True) 
nbr_zero_par_station

nombre des stations :  126


,Station,Precipitation_zero,Pourcentage par rapport au nbr zero total%
125,ZOUEM SM,33087,1.572808
78,MUNCHAR ECOLE,30560,1.452686
46,FEIJA EL SM,30507,1.450166
106,SK EL ARBA (JENDOUBA,29896,1.421122
76,MONTARNAUD 1,29048,1.380812
...,...,...,...
120,TOUKEBER,6673,0.317205
51,GDOUD UCP,6669,0.317014
40,DJEBEL KBOUCH PF,6390,0.303752
71,LAAROUSSA AGRICOLE,6171,0.293342


In [107]:
#count_precepitation_par_station
cpps=df.groupby('Station')['Precipitation'].apply(lambda x: (x).count()).reset_index(name='count')

cpps[cpps["Station"]=="TOUKEBER"]["count"].values[0]
cpps


,Station,count
0,AIN BEYA OUED RHEZAL,18466
1,AIN DEBBA,16537
2,AIN GUESIL 1,17708
3,AIN HAMRAYA,22286
4,AIN JEMMALA,15314
...,...,...
121,UTIQUE OMVVM,9129
122,ZAAFRANE UCP,19385
123,ZEBIDA UCP ENNAJAT,11657
124,ZOUARINE GARE,18474


In [112]:
#tableu % des valuer 0 pour chaque station max 93.214286 % des valeur de station SENED EL HADDAD sont des 0 
#et 73.485930 % des valeurs de stations  TIBAR SM	sont des 0
nbr_zero_par_station["Pourcentage par rapport au nbr zero station%"] = nbr_zero_par_station.apply(lambda x:percent(x["Precipitation_zero"],cpps[cpps["Station"]==x["Station"]]["count"].values[0]) ,axis=1)
nbr_zero_par_station.sort_values(by=["Pourcentage par rapport au nbr zero station%"],ascending=False, inplace=True) 
nbr_zero_par_station

,Station,Precipitation_zero,Pourcentage par rapport au nbr zero total%,Pourcentage par rapport au nbr zero station%
93,SENED EL HADDAD,16704,0.794033,93.214286
55,HIR MAZDOUR,8780,0.417362,93.126856
39,DJ BOU LAHNECH (E.PR,9568,0.454820,92.399807
2,AIN GUESIL 1,16181,0.769172,91.376779
80,NMAIRIA,6987,0.332131,91.107054
...,...,...,...,...
1,AIN DEBBA,12529,0.595573,75.763440
65,KEF.B.I.R.H,9399,0.446786,75.700709
46,FEIJA EL SM,30507,1.450166,75.051663
25,BEN METIR 2 SM,18370,0.873228,74.075568


In [113]:
#ordre Croissant
nbr_zero_par_station.sort_values(by=["Pourcentage par rapport au nbr zero station%"],ascending=True, inplace=True) 
nbr_zero_par_station

,Station,Precipitation_zero,Pourcentage par rapport au nbr zero total%,Pourcentage par rapport au nbr zero station%
118,TIBAR SM,28830,1.370449,73.485930
25,BEN METIR 2 SM,18370,0.873228,74.075568
46,FEIJA EL SM,30507,1.450166,75.051663
65,KEF.B.I.R.H,9399,0.446786,75.700709
1,AIN DEBBA,12529,0.595573,75.763440
...,...,...,...,...
80,NMAIRIA,6987,0.332131,91.107054
2,AIN GUESIL 1,16181,0.769172,91.376779
39,DJ BOU LAHNECH (E.PR,9568,0.454820,92.399807
55,HIR MAZDOUR,8780,0.417362,93.126856
